# Global mean Sea Surface Temperatures

**sstm** - SST monthly mean (time: 492,lat: 720,lon: 1440)

**sstm_WGlobalmean** - SST weighted global mean (time)

**sstm_Clim** - monthly climatology (month: 12lat: 720lon: 1440)

**sstm_anom** - anomaly from monthly Climatology (time: 492, lat: 720,lon: 1440)

**sstm_anom_WGlobalmean** - SST anomlay weighted global mean (time: 492)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import cartopy
import cartopy.crs as ccrs
plt.rcParams['figure.figsize'] = (10, 4)

#### Settings

In [ ]:
#To compute de climatoloy
year1='1982'
year2='1992'

## Load data

In [ ]:
base_file = '/data/shareddata/Satelite/noaa.oisst.v2.highres/NC/sst.day.mean'
files = [f'{base_file}.{year}.nc' for year in range(1982, 2023)]
sst = xr.open_mfdataset(files)
sst = sst.sst

### Resample data to 1 month

In [ ]:
# Monthly mean
sstm = sst.resample(time='1M').mean(dim='time',skipna=True).load()
sstm_SH = sstm.sel( lat=slice( -90, 0) ).load()
sstm_NH = sstm.sel( lat=slice( 0, 90) ).load()

## Global mean
(weigthed and unweigthed)

In [ ]:
# Calculate global mean weigthtened
# For a rectangular grid the cosine of the latitude is proportional to the grid cell area.
weights = np.cos(np.deg2rad(sstm.lat))
weights.name = "weights"
sstm_weighted = sstm.weighted(weights)
sstm_WGlobalmean = sstm_weighted.mean(("lon", "lat"),skipna=True).load()

## Seasonal cycle
Create seasonal climatology from 1982 to 1992 and the asociated anomaly

In [ ]:
#Create monthly climatology
sstm_Clim = sstm.sel(time=slice(year1,year2)).groupby('time.month').mean(dim='time').load();

In [ ]:
#Create anomaly
sstm_anom = sstm.groupby('time.month') - sstm_Clim
sstm_anom.load();

## Global mean anomaly

In [ ]:
# Calculate global mean weigthtened
# For a rectangular grid the cosine of the latitude is proportional to the grid cell area.
weights = np.cos(np.deg2rad(sstm.lat))
weights.name = "weights"
sstm_anom_weighted = sstm_anom.weighted(weights)
sstm_anom_WGlobalmean = sstm_anom_weighted.mean(("lon", "lat"),skipna=True).load()
sstm_anom_WGlobalmean_rolling = sstm_anom_WGlobalmean.rolling(time=12,center=True).mean()

In [ ]:
sstm_WGlobalmean_Y = sstm_WGlobalmean.resample(time='1Y').mean(dim='time',skipna=True)
sstm_anom_WGlobalmean_Y = sstm_anom_WGlobalmean.resample(time='1Y').mean(dim='time',skipna=True)

In [ ]:
fig, ax = plt.subplots(1, 1 , figsize=(10,4))
ax.plot(sstm_WGlobalmean.time,sstm_WGlobalmean)
ax.plot(sstm_WGlobalmean_Y.time,sstm_WGlobalmean_Y,'-b')
ax.grid()
ax.legend()
ax.set_title('Mean global Sea Surface Temperature')

In [ ]:
fig, ax = plt.subplots(1, 1 , figsize=(10,4))
ax.plot(sstm_anom_WGlobalmean.time,sstm_anom_WGlobalmean,'b')
ax.plot(sstm_anom_WGlobalmean_Y.time,sstm_anom_WGlobalmean_Y,'-b')
ax.plot(sstm_anom_WGlobalmean_rolling.time,sstm_anom_WGlobalmean_rolling,'r')

ax.plot(sstm_anom_WGlobalmean.time.isel(sstm_anom_WGlobalmean.argmax(...)),
        sstm_anom_WGlobalmean.isel(sstm_anom_WGlobalmean.argmax(...)),'ro')
ax.plot(sstm_anom_WGlobalmean.time.isel(sstm_anom_WGlobalmean.argmin(...)),
        sstm_anom_WGlobalmean.isel(sstm_anom_WGlobalmean.argmin(...)),'bo')

ax.grid()
ax.legend()
ax.set_title('Mean global Sea Surface Temperature anomaly\n Anomaly computed from the 1982-1992 period');



In [ ]:
fig, ax = plt.subplots(1, 1 , figsize=(5,5))
ax.contourf(sstm_Clim.mean(dim='lon').month,
                 sstm_Clim.mean(dim='lon').lat,
                 sstm_Clim.mean(dim='lon').transpose(),levels=12, vmin=-2, vmax=30)
ax.set_ylim(-78,78)
ax.grid()
ax.set_title('Seasonal Climatoloy (' + year1 + '-' + year2 + ')')
ax.set_ylabel('Latitude') 
ax.set_xlabel('Month')
ax.grid()


## Hemisphere analysis

In [ ]:
# Calculate NH mean weigthtened
weights = np.cos(np.deg2rad(sstm_NH.lat))
weights.name = "weights"
sstm_NH_weighted = sstm_NH.weighted(weights)
sstm_NH_WGlobalmean = sstm_NH_weighted.mean(("lon", "lat"),skipna=True).load()

#Compute climatology
sstm_NH_Clim = sstm_NH.sel(time=slice(year1,year2)).groupby('time.month').mean(dim='time').load();

#Create anomaly
sstm_NH_anom = sstm_NH.groupby('time.month') - sstm_Clim
sstm_NH_anom.load();

sstm_NH_anom_weighted = sstm_NH_anom.weighted(weights)
sstm_NH_anom_WGlobalmean = sstm_NH_anom_weighted.mean(("lon", "lat"),skipna=True).load()

sstm_NH_anom_WGlobalmean_rolling = sstm_NH_anom_WGlobalmean.rolling(time=12,center=True).mean()

In [ ]:
# Calculate NH mean weigthtened
weights = np.cos(np.deg2rad(sstm_SH.lat))
weights.name = "weights"
sstm_SH_weighted = sstm_SH.weighted(weights)
sstm_SH_WGlobalmean = sstm_SH_weighted.mean(("lon", "lat"),skipna=True).load()

#Compute climatology
sstm_SH_Clim = sstm_SH.sel(time=slice(year1,year2)).groupby('time.month').mean(dim='time').load();

#Create anomaly
sstm_SH_anom = sstm_SH.groupby('time.month') - sstm_Clim
sstm_SH_anom.load();

sstm_SH_anom_weighted = sstm_SH_anom.weighted(weights)
sstm_SH_anom_WGlobalmean = sstm_SH_anom_weighted.mean(("lon", "lat"),skipna=True).load()

sstm_SH_anom_WGlobalmean_rolling = sstm_SH_anom_WGlobalmean.rolling(time=12,center=True).mean()

In [ ]:
fig, ax = plt.subplots(3, 2 , figsize=(14,12))

ax[0,0].plot(sstm_NH_WGlobalmean.time,sstm_NH_WGlobalmean) 
ax[0,0].set_title('Northern hemisphere SST mean')
ax[0,0].set_ylim(14,24)
ax[0,0].grid()
ax[0,1].plot(sstm_SH_WGlobalmean.time,sstm_SH_WGlobalmean) 
ax[0,1].set_title('Southern hemisphere SST mean')
ax[0,1].set_ylim(14.7,23.2)
ax[0,1].grid()

ax[1,0].plot(sstm_NH_anom_WGlobalmean.time,sstm_NH_anom_WGlobalmean) 
ax[1,0].set_title('Northern hemisphere SST mean anomlay')
ax[1,0].set_ylim(-0.25,1.1)
ax[1,0].grid()
ax[1,1].plot(sstm_SH_anom_WGlobalmean.time,sstm_SH_anom_WGlobalmean) 
ax[1,1].set_title('Southern hemisphere SST mean anomlay')
ax[1,1].set_ylim(-0.25,1.1)
ax[1,1].grid()

ax[2,0].contourf(sstm_NH_Clim.mean(dim='lon').month,
                 sstm_NH_Clim.mean(dim='lon').lat,
                 sstm_NH_Clim.mean(dim='lon').transpose(),levels=12, vmin=-2, vmax=30)
ax[2,0].set_ylim(0,80)
ax[2,0].grid()
ax[2,0].set_title('Seasonal Climatoloy (' + year1 + '-' + year2 + ')')
ax[2,0].set_ylabel('Latitude') 
ax[2,0].set_xlabel('Month')
ax[2,0].grid()
ax[2,1].contourf(sstm_SH_Clim.mean(dim='lon').month,
                 sstm_SH_Clim.mean(dim='lon').lat,
                 sstm_SH_Clim.mean(dim='lon').transpose(),levels=12, vmin=-2, vmax=30)

ax[2,1].set_ylim(-80,0)
ax[2,1].grid()
ax[2,1].set_title('Seasonal Climatoloy (' + year1 + '-' + year2 + ')')
ax[2,1].set_ylabel('Latitude') 
ax[2,1].set_xlabel('Month')




#ax[0].set_title('Mean global Sea Surface Temperature')


In [ ]:
sstm.to_netcdf('sstm.nc')
sstm_WGlobalmean.to_netcdf('sstm_WGlobalmean.nc')
sstm_Clim.to_netcdf('sstm_Clim.nc')
sstm_anom.to_netcdf('sstm_anom.nc')
sstm_anom_WGlobalmean.to_netcdf('sstm_anom_WGlobalmean.nc')

## Annex A

### Compare weigthtened and unweigthtened global means

In [ ]:
#Calculate global mean unweigthtened
sstm_UWGlobalmean = sstm.mean(dim='lon',skipna=True).mean(dim='lat',skipna=True).load()

In [ ]:
sstm_WGlobalmean.plot(label="weighted")
sstm_UWGlobalmean.plot(label="unweighted")
plt.grid()
plt.legend()

### Canary Islands example

In [ ]:
sstm_NorteTenerife = sstm.sel(lon=360-16.1188,lat=28.5559,method='nearest').load()
sstm_anom_NorteTenerife = sstm_anom.sel(lon=345,lat=29,method='nearest').load()
sstm_anom_NorteTenerife_rolling = sstm_anom.sel(lon=345,lat=29,method='nearest').rolling(time=12,center=True).mean()
sstm_NorteTenerife_Y = sstm_NorteTenerife.resample(time='1Y').mean(dim='time',skipna=True)

In [ ]:
fig, ax = plt.subplots(1, 1 , figsize=(14,10))
ax.plot(sstm_NorteTenerife.time,sstm_NorteTenerife)
ax.grid()
ax.legend()
ax.set_title('Mean global Sea Surface Temperature')

In [ ]:
fig, ax = plt.subplots(1, 1 , figsize=(14,10))
ax.plot(sstm_NorteTenerife_Y.time,sstm_NorteTenerife_Y)
ax.grid()
ax.legend()
ax.set_title('Mean global Sea Surface Temperature')
ax.plot(sstm_NorteTenerife_Y.time.isel(sstm_NorteTenerife_Y.argmax(...)),
           sstm_NorteTenerife_Y.isel(sstm_NorteTenerife_Y.argmax(...)),'ro')
ax.plot(sstm_NorteTenerife_Y.time.isel(sstm_NorteTenerife_Y.argmin(...)),
           sstm_NorteTenerife_Y.isel(sstm_NorteTenerife_Y.argmin(...)),'bo')

In [ ]:
fig, ax = plt.subplots(1, 1 , figsize=(14,8))
ax.plot(sstm_anom_NorteTenerife.time,sstm_anom_NorteTenerife,'b')
ax.plot(sstm_anom_NorteTenerife_rolling.time,sstm_anom_NorteTenerife_rolling,'r')
ax.plot(sstm_anom_NorteTenerife.time.isel(sstm_anom_NorteTenerife.argmax(...)),
           sstm_anom_NorteTenerife.isel(sstm_anom_NorteTenerife.argmax(...)),'ro')
ax.plot(sstm_anom_NorteTenerife.time.isel(sstm_anom_NorteTenerife.argmin(...)),
           sstm_anom_NorteTenerife.isel(sstm_anom_NorteTenerife.argmin(...)),'bo')
ax.legend()
ax.grid()
ax.set_title('Mean global Sea Surface Temperature anomaly');

In [ ]:
sstm_anom_NorteTenerife.time.isel(sstm_anom_NorteTenerife.argmax(...)).dt.strftime('%Y')

In [ ]:
sstm.to_netcdf('noaa.oisst.v2.highres.sstm.nc')
sstm_WGlobalmean.to_netcdf('noaa.oisst.v2.highres.sstmWGM.nc')
sstm_Clim.to_netcdf('noaa.oisst.v2.highres.sstmC.nc')
sstm_anom.to_netcdf('noaa.oisst.v2.highres.sstmA.nc')
sstm_anom_WGlobalmean.to_netcdf('noaa.oisst.v2.highres.sstmAWGM.nc')

In [ ]:
import pandas as pd
time = pd.date_range("2000-01-01", "2004-12-31")
base = xr.DataArray(data=np.ones((time.size, 3, 2)),
                    dims=("time", "lat", "lon"),
                    coords={"time": time,"lat": [1, 2, 3],"lon": [0.5, 1.5],},)

In [ ]:
seasonality = xr.DataArray(
      data=np.sin((time.dayofyear / 365.0) * (2 * np.pi)),
      coords={"time": time},
      dims=["time"],)

In [ ]:
trend = xr.DataArray(
      data=(time.year - 2000) * 0.1,
      coords={"time": time},
      dims=["time"])

In [ ]:
da = base + seasonality + trend

In [ ]:
location = da.isel(lat=0, lon=0)
location

In [ ]:
dataframe = location.groupby(da["time"].dt.dayofyear).min().drop(["lat",
  "lon"]).to_dataframe(name="min")
dataframe["max"] = location.groupby(da["time"].dt.dayofyear).max().values

In [ ]:
dataframe

In [ ]:
for year, yearda in location.groupby(location["time"].dt.year):
      dataframe[year] = pd.Series(index=yearda["time"].dt.dayofyear,
  data=yearda.values)

In [ ]:
yearda

In [ ]:
location2=sstm_anom_WGlobalmean

In [ ]:
dataframe2 = location2.groupby(location2.time.dt.month).min().to_dataframe(name="min")
dataframe2["max"] = location2.groupby(location2.time.dt.month).max().values

In [ ]:
for year, yearda in location2.groupby(location2.time.dt.year):
    dataframe2[year] = pd.Series(index=yearda["time"].dt.month,
                         data=yearda.values)
    
    

In [ ]:
fig, ax = plt.subplots()
ax.fill_between(x=dataframe2.index, y1=dataframe2["min"], y2=dataframe2["max"],
alpha=0.5, color="orange")
dataframe2.plot(ax=ax)

In [ ]:
location2.time.dt.dayofyear.sel(time='1992').values